In [ ]:
# Baseline LSTM for Stock Market Prediction (multiple tickers, OHLC only, no scaling)

import math
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

np.random.seed(42)
tf.random.set_seed(42)

# ----------------- Config -----------------
TICKERS = ["AAPL", "MSFT", "GOOG"]   # List of tickers
START = "2015-01-01"
END = "2025-01-01"
SEQ_LEN = 21
EPOCHS = 30
BATCH_SIZE = 32
# ------------------------------------------

def create_sequences(arr, seq_len, target_col_idx=3):
    X, y = [], []
    for i in range(len(arr) - seq_len):
        X.append(arr[i:i+seq_len])
        y.append(arr[i+seq_len, target_col_idx])
    return np.array(X), np.array(y)

def build_lstm(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=False, input_shape=input_shape),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

for ticker in TICKERS:
    print(f"\n===== Running Baseline LSTM for {ticker} =====")

    # 1) Download data
    df = yf.download(ticker, start=START, end=END, progress=False)
    df = df[['Open', 'High', 'Low', 'Close']].round(2).dropna()

    # 2) Sequence creation
    values = df.values
    X, y = create_sequences(values, SEQ_LEN, 3)

    # Train-test split
    split = int(len(X) * 0.8)
    X_train, y_train = X[:split], y[:split]
    X_test, y_test = X[split:], y[split:]

    # 3) Model
    model = build_lstm((SEQ_LEN, X.shape[2]))
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

    # 4) Evaluate
    pred = model.predict(X_test).ravel()
    rmse = math.sqrt(mean_squared_error(y_test, pred))
    mae = mean_absolute_error(y_test, pred)
    mape = mean_absolute_percentage_error(y_test, pred)

    print(f"{ticker} → RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.4f}")



===== Running Baseline LSTM for AAPL =====


/tmp/ipython-input-4258611522.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
AAPL → RMSE: 96.8514, MAE: 92.7932, MAPE: 0.4805

===== Running Baseline LSTM for MSFT =====


/tmp/ipython-input-4258611522.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
MSFT → RMSE: 270.8924, MAE: 263.7478, MAPE: 0.7149

===== Running Baseline LSTM for GOOG =====


/tmp/ipython-input-4258611522.py:44: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=START, end=END, progress=False)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
GOOG → RMSE: 65.8385, MAE: 59.8252, MAPE: 0.3969
